In [2]:
import pandas as pd

In [3]:
from collections import Counter, defaultdict

In [4]:
files = ["auction.csv",
        "kaiin.csv",
        "nyuuka_oshirase.csv",
        "shudounyuusatsu.csv",
        "watchlist.csv",#お気に入り追加データ
        ]
#入札と落札があるが、落札は入札に内包されるはずなので、入札だけ考えれば良いかと

In [5]:
df_kaiin = pd.read_csv("/Users/yuki/Desktop/nishika_competition/brandear_competition/kaiin.csv")

In [6]:
df_auction = pd.read_csv("/Users/yuki/Desktop/nishika_competition/brandear_competition/auction.csv")

In [7]:
df_submission = pd.read_csv("/Users/yuki/Desktop/nishika_competition/brandear_competition/sample_submission.csv")

In [8]:
df_nyuusatsu = pd.read_csv("/Users/yuki/Desktop/nishika_competition/brandear_competition/shudounyuusatsu.csv")

In [9]:
df_watchlist = pd.read_csv("/Users/yuki/Desktop/nishika_competition/brandear_competition/watchlist.csv")

In [50]:
#test_kaiin_id =  [df_submission["KaiinID"].iloc[int(i*20)] for i in range(int(len(df_submission["KaiinID"])/20))]

In [10]:
log = pd.read_csv("search_log.tsv",delimiter='\t')

/Users/yuki/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#9月17日から23日までのデータを訓練データとする

In [51]:
df_nyuusatsu_train = df_nyuusatsu.iloc[-12135:-1] #正解が含まれているから使わないよ。
df_nyuusatsu_train = df_nyuusatsu.reset_index(drop=True)

In [12]:
kaiin_dict = defaultdict(int)
for d in df_nyuusatsu_train["KaiinID"]:
    kaiin_dict[int(d)] += 1

In [29]:
train_kaiin_id = []
for k in kaiin_dict:
    if kaiin_dict[k] <= 60:
        train_kaiin_id.append(k)

In [30]:
len(train_kaiin_id)

2295

In [31]:
df_watchlist.iloc[775444]

KaiinID                     146248
AuctionID                   887828
TourokuDate    2019-09-17 00:00:03
SakujoFlag                       0
Name: 775444, dtype: object

In [32]:
df_watchlist.iloc[860508]

KaiinID                      72159
AuctionID                   750614
TourokuDate    2019-09-23 23:59:59
SakujoFlag                       1
Name: 860508, dtype: object

In [49]:
df_watchlist_x = pd.concat([df_watchlist.iloc[0:775444], df_watchlist.iloc[860509:-1]])
df_watchlist_x = df_watchlist_x.reset_index(drop=True)

In [40]:
df_watchlist_y = df_watchlist.iloc[775444:860509]
df_watchlist_y.head(1)

,KaiinID,AuctionID,TourokuDate,SakujoFlag
775444,146248,887828,2019-09-17 00:00:03,0


In [43]:
df_watchlist_y = df_watchlist.iloc[775444:860509]
df_watchlist_y = df_watchlist_y.reset_index(drop=True)
l_y = []
for i, d in enumerate(df_watchlist_y["KaiinID"]):
    if d in train_kaiin_id:
        l_y.append([d, df_watchlist_y["AuctionID"][i],
                   df_watchlist_y["TourokuDate"][i], 1])
df_watchlist_y = pd.DataFrame(l_y, columns=["KaiinID", "AuctionID", "Date", "N"])

In [57]:
df_watchlist_y.head(1)

,KaiinID,AuctionID,Date,N
0,146248,887828,2019-09-17 00:00:03,1


In [55]:
ll_y = []
for i, d in enumerate(df_nyuusatsu_train["KaiinID"]):
    if d in train_kaiin_id:
        ll_y.append([d, df_nyuusatsu_train["AuctionID"][i],
                     df_nyuusatsu_train["ShudouNyuusatsuDate"][i],
                     2])
df_nyuusatsu_y = pd.DataFrame(ll_y, columns=["KaiinID","AuctionID","Date","N"])

In [56]:
df_nyuusatsu_y.head(1)

,KaiinID,AuctionID,Date,N
0,258477,636604,2018-09-01 19:46:13,2


In [61]:
train_y = pd.concat([df_nyuusatsu_y, df_watchlist_y])
train_y = train_y.reset_index(drop=True)

In [64]:
len(train_y)

262316

In [76]:
len(df_auction)

4109559

In [ ]:

feature_y = []
for j, t in enumerate(df_auction["AuctionID"]):
    for i, d in enumerate(train_y["AuctionID"]):
        if d == t:
            feature_y.append([d, train_y["KaiinID"][i],
                            train_y["Date"][i],
                            train_y["N"][i],
                          df_auction["ShouhinShubetsuID"][j],
                           df_auction["ConditionID"][j],
                           df_auction["BrandID"][j],
                           df_auction["GenreID"][j],
                           df_auction["GenreGroupID"][j],
                           df_auction["LineID"][j],
                           df_auction["ColorID"][j],
                           df_auction["DanjobetsuID"][j],
                           df_auction["SankouKakaku"][j]]
                          )
            continue
    if j == 1000000:
        print(1000000)

In [65]:
df_nyuusatsu_train_2 = df_nyuusatsu.iloc[0:len(df_nyuusatsu)-12136]

In [66]:
df_nyuusatsu_train_2.head(1)

,AuctionID,KaiinID,ShudouNyuusatsuDate,Kingaku,Suuryou,SokketsuFlag,SakujoFlag
0,3744587,227198,2018-09-01 21:40:44,18000,1,0,1


In [67]:
train = []
for i, d in enumerate(df_nyuusatsu_train_2["KaiinID"]):
    if d in train_kaiin_id:
        train.append([d, df_nyuusatsu_train_2["AuctionID"][i], df_nyuusatsu_train_2["ShudouNyuusatsuDate"][i], 2])
for i, d in enumerate(df_watchlist_x["KaiinID"]):
    if d in train_kaiin_id:
        train.append([d, df_watchlist_x["AuctionID"][i], df_watchlist_x["TourokuDate"][i], 1])
train_x = pd.DataFrame(train, columns=["KaiinID","AuctionID", "Date", "N"])

In [69]:
train_x.to_csv("train_x.csv",index=False)

In [81]:
train_x.iloc[300000]

KaiinID                    89469
AuctionID                1664323
Date         2019-09-02 21:39:15
Name: 300000, dtype: object

In [ ]:
feature = []
for j, t in enumerate(df_auction["AuctionID"]):
    for i, d in enumerate(train_x["AuctionID"]):
        if d == t:
            feature.append([d, train_x["KaiinID"][i],
                            train_x["Date"][i],
                          df_auction["ShouhinShubetsuID"][j],
                           df_auction["ConditionID"][j],
                           df
                            _auction["BrandID"][j],
                           df_auction["GenreID"][j],
                           df_auction["GenreGroupID"][j],
                           df_auction["LineID"][j],
                           df_auction["ColorID"][j],
                           df_auction["DanjobetsuID"][j],
                           df_auction["SankouKakaku"][j]]
                          )
            continue

In [75]:
len(train_x)

1176689

In [76]:
len(df_auction)

4109559

In [38]:
train_x_2 = df.DataFrame(feature, columns=["AuctionID","KaiinID","ShouhinShubetsuID","ConditionID",
                                           "BrandID","GenreID","GenreGroupID","LineID","ColorID","DanjobetsuID",
                                          "SankouKakaku"])

,AuctionID,ShouhinShubetsuID,ShouhinID,SaishuppinKaisuu,ConditionID,BrandID,GenreID,GenreGroupID,LineID,ColorID,DanjobetsuID,SankouKakaku,CreateDate
0,4066465,1,620477,334,6,494,321,312,0,15,0,4000,2019-07-27 09:31:34


In [34]:
df_watchlist.head(3)

,KaiinID,AuctionID,TourokuDate,SakujoFlag
0,231713,1036190,2018-09-01 00:00:16,1
1,225585,4043640,2018-09-01 00:00:34,1
2,75217,3100882,2018-09-01 00:00:44,1


In [57]:
df_nyuusatsu.iloc[600000]

AuctionID                          3368992
KaiinID                             129830
ShudouNyuusatsuDate    2019-09-16 14:29:05
Kingaku                              11400
Suuryou                                  1
SokketsuFlag                             0
SakujoFlag                               1
Name: 600000, dtype: object